## Evaluate the model

Choose the K in TOP K accuracy

In [ ]:
def get_random_userid():
    # pick 1 random user in the training data at random
    user_id_eval = np.random.randint(num_users)
    # user_id_eval_df = users_mapping[user_id_eval]
    return user_id_eval

user_id_eval = get_random_userid()
user_id_eval

2800

In [ ]:
def get_y_true(user_id):
    # Stacking x_train and y_train
    y_true_tmp = np.hstack((x_train, y_train))
    # keep only the recommended games
    y_true_tmp = y_true_tmp[y_true_tmp[:,-1] == 1]
    # filtering for the user id
    y_true = y_true_tmp[y_true_tmp[:,0] == user_id][:,1]
    return y_true
    
y_true = get_y_true(user_id_eval)
y_true

array([   19,    20,   724,  1231,  1331,  1347,  1459,  1551,  1899,
        2009,  2156,  2264,  2307,  4271,  6635,  6712,  6739,  9013,
       10739, 12824, 14127, 14243, 14252, 14264, 14942, 17637, 17638,
       17800])

In [ ]:
def get_user_games(user_id):
    # Generate a list of all games for a specific user
    games = list(games_mapping.keys())
    games2pred = np.reshape(games, (-1,1))
    user_game_array_eval = np.hstack(([[user_id]] * len(games), games2pred))
    return user_game_array_eval

user_games = get_user_games(user_id_eval)
user_games

array([[ 2800,     0],
       [ 2800,     1],
       [ 2800,     2],
       ...,
       [ 2800, 18889],
       [ 2800, 18890],
       [ 2800, 18891]])

In [ ]:
def get_y_pred(user_games_array, k):
    preds = model.predict(user_games_array)
    user_games_preds = np.hstack((user_games_array, preds))
    sorted_indices = np.argsort(-user_games_preds[:, -1])
    y_pred = user_games_preds[sorted_indices][:k, :]
    y_pred = y_pred[:,1].astype('int32')
    return y_pred

y_pred = get_y_pred(user_games, K)
y_pred

591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step


array([  412, 12824,  3999,  4048, 11343,  1143,  1468,  3953,  1166,
         212])

In [ ]:
def eval_k(y_true, y_pred):
    items_relevant_user = len(y_true)
    rec_items_relevant = len(set(y_true).intersection(set(y_pred)))
    recall = rec_items_relevant / items_relevant_user
    precision = rec_items_relevant / K
    return recall, precision

eval_k(y_true, y_pred)

(0.03571428571428571, 0.1)

In [ ]:
def eval_k_avg(n_users, K):
    precision = []
    recall = []
    for i in range(n_users):
        user_id_eval = get_random_userid()
        y_true = get_y_true(user_id_eval)
        user_games = get_user_games(user_id_eval)
        y_pred = get_y_pred(user_games, K)
        rcl, prc = eval_k(y_true, y_pred)
        precision.append(prc)
        recall.append(rcl)
    avg_precision = np.mean(precision)
    avg_recall = np.mean(recall)
    return avg_precision, avg_recall

avg_precision, avg_recall = eval_k_avg(10, 5)

print("K",K)
print("Number of users:", num_users)
print("Average Precision", '{0:,.0f}'.format(avg_precision))
print("Average Precision", '{0:,.0f}'.format(avg_recall))

591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step


(0.030000000000000006, 0.008292282430213464)

In [ ]:
def eval_hit_ratio(K, n_users):
    counter = 0
    for user_game in x_test[:n_users,:]:
        user_games = get_user_games(user_game[0])
        y_pred = get_y_pred(user_games, K)
        if(user_game[1] in y_pred):
            counter += 1
    return counter / n_users

print("Hit Ratio:",  '{0:,.0f}'.format(eval_hit_ratio(30, 30)))

548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 919us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 941us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 973us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 892us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 906us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 951us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 989us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step
548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
548/548 ━━━━━━━━━━━━━

0.06666666666666667